In [1]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
import geopandas as gpd
from tqdm import tqdm
import shap
from sklearn.metrics import r2_score
import pandas as pd

C:\Users\Holger\micromamba\envs\agile2024\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Search for hyperparameters
def search_hyperparams(X, y, random_state):
    
    # Number of trees in random forest
    n_estimators = list(np.linspace(start=10, stop=100, num=10, dtype=int))
    
    # Number of features to consider at every split
    max_features = [1, "sqrt", "log2"]
    
    # Maximum number of levels in tree
    max_depth = list(np.linspace(start=10, stop=100, num=10, dtype=int))
    max_depth.append(None)
    
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    
    # Create dictionary from parameters
    param_distributions = {
        "n_estimators": n_estimators,
        "max_features": max_features,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf,
        "bootstrap": bootstrap
    }
    
    # Perform search for hyperparameters
    estimator = RandomForestRegressor()
    rf_random = RandomizedSearchCV(
        estimator=estimator, param_distributions=param_distributions, n_iter=100, verbose=2, random_state=random_state,
        n_jobs=-1
    )
    rf_random.fit(X, y)
    
    # Get best parameters
    params = rf_random.best_params_
    params["bootstrap"] = True
    params["oob_score"] = True
    
    return params

In [3]:
# Run experiment
def run_experiment(fp, test_size, n_features_to_keep, random_state):
    
    # Read observation data
    obs_data = gpd.read_file(fp)
    
    # Extract features and target
    X = obs_data.iloc[:, 3:-1]
    y = obs_data["obs_value"]
    
    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Perform search for hyperparameters
    params = search_hyperparams(X_train, y_train, random_state)
    
    # RF regressor
    regressor = RandomForestRegressor()
    
    # Set hyperparameters
    regressor.set_params(**params)
    
    # Fit model
    regressor.fit(X_train, y_train)

    # Predict
    Y_train_pred = regressor.predict(X_train)
    Y_test_pred = regressor.predict(X_test)
    
    # Calculate SHAP values
    explainer = shap.TreeExplainer(regressor)
    shap_values = shap.TreeExplainer(regressor).shap_values(X_train)
    
    # Create DataFrame of absolute mean SHAP values
    abs_mean_shap_df = pd.DataFrame(
        list(zip(X_test.columns, np.abs(shap_values).mean(0))), columns=["feature", "abs_mean_shap"]
    )
    
    # Sort based on absolute mean SHAP values
    abs_mean_shap_df = abs_mean_shap_df.sort_values(by=["abs_mean_shap"], ascending=False).reset_index(drop=True)
    
    # List of most important features
    top_features = abs_mean_shap_df["feature"].head(n_features_to_keep).to_list()
    
    # Generate new training and test feature sets
    X_train_reduced = X_train[top_features]
    X_test_reduced = X_test[top_features]
    
    # Perform search for hyperparameters
    params_reduced = search_hyperparams(X_train_reduced, y_train, random_state)
    
    # RF regressor
    regressor_reduced = RandomForestRegressor()
    
    # Set hyperparameters
    regressor_reduced.set_params(**params_reduced)
    
    # Fit model
    regressor_reduced.fit(X_train_reduced, y_train)

    # Predict
    Y_train_pred_reduced = regressor_reduced.predict(X_train_reduced)
    Y_test_pred_reduced = regressor_reduced.predict(X_test_reduced)
    
    # Calculate results
    results = {
        "test_size": test_size,
        "n_features": len(X_train.columns),
        "r2_train": regressor.score(X_train, y_train),
        "r2_test": r2_score(y_test, Y_test_pred),
        "n_features_reduced": len(X_train_reduced.columns),
        "r2_train_reduced": regressor_reduced.score(X_train_reduced, y_train),
        "r2_test_reduced": r2_score(y_test, Y_test_pred_reduced)
    }
    
    return pd.DataFrame([results])

In [4]:
%%time

# Aggregation function list
agg_func_list = ["mean", "median"]

# Test set size list
test_size_list = [0.1, 0.2, 0.3, 0.4, 0.5]

# Number of features to keep based on SHAP values
n_features_to_keep_list = [10, 15, 20]

# Number of iterations
n_iterations = 10

# Random state
random_state = 1

for agg_func in tqdm(agg_func_list, total=len(agg_func_list), position=0, leave=True):
    
    for test_size in test_size_list:
        
        for n_features_to_keep in n_features_to_keep_list:
            
            # Path to input data
            fp = f"agile2024_tn_{agg_func}_obs_sites.gpkg"
            
            # Iterate experiments and collect results
            results_df_list = []
            for i in range(1, n_iterations + 1):
                results_df = run_experiment(fp, test_size, n_features_to_keep, random_state)
                results_df.insert(0, "iteration", i)
                results_df.insert(1, "agg_func", agg_func)
                results_df["r2_test_reduced_gain"] = results_df["r2_test_reduced"] - results_df["r2_test"]
                results_df_list.append(results_df)
                
            # Concatenate results and write to CSV
            out_df = pd.concat(results_df_list).reset_index(drop=True)
            out_fp = f"experiments/agg_func_{agg_func}_test_size_{test_size}_n_features_to_keep_{n_features_to_keep}.csv"
            out_df.to_csv(out_fp, index=False)
            print(f"File {out_fp} has been created!")

  0%|                                                                                                                                                                                                                 | 0/2 [00:00<?, ?it/s]

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\Holger\micromamba\envs\agile2024\Lib\site-packages\sklearn\ensemble\_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\Holger\micromamba\envs\agile2024\Lib\site-packages\sklearn\ensemble\_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


...

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\Holger\micromamba\envs\agile2024\Lib\site-packages\sklearn\ensemble\_forest.py:615: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [3:49:18<00:00, 6879.47s/it]

File experiments/agg_func_median_test_size_0.5_n_features_to_keep_20.csv has been created!
CPU times: total: 38min 9s
Wall time: 3h 49min 18s
